# One Data Extract

TJ Kim
8/5/20

Extract from csv file ONE SIMULATOR data and group mobility.

The columns are as follows [time (s), node id (int), x coor (meter), y coor(meters), z coor (meters)]

The simulation takes place in a x=3925m and y = 2750m space.

The goal is to extract these columns per user and apply to stochastic movement.

In [86]:
import numpy as np

def get_one_sim_usr():
    """
    Get users and their average speeds in a list in onesim
    """
    # Import Data and eliminate z axis
    one_sim_data = np.genfromtxt('ONE_SIM.csv', delimiter=',')
    one_sim_data = one_sim_data[:,:4]

    # Data polishing
    meter2mile = 0.000621371
    # Convert Time from seconds --> 5 min intervals
    one_sim_data[:,0] *= (1/300)
    one_sim_data[:,0] = np.floor(one_sim_data[:,0])
    # Convert distance from meters --> Miles
    one_sim_data[:,2] *= meter2mile
    one_sim_data[:,3] *= meter2mile

    # Data Dividing and sampling
    ids = one_sim_data[:,1]
    unique_ids = np.unique(ids) 

    one_users = {}
    for idx in unique_ids:
        # Take id locations and take out relevant id data
        curr_locs = np.where(ids == idx)[0]
        start_loc, end_loc = curr_locs[0], curr_locs[-1]
        one_sim_data_u = one_sim_data[start_loc:end_loc,:]

        # Sampling (time seems to go from zero to 36)
        times = np.arange(0,37)
        sampled_data = np.zeros((times.shape[0],5))

        for t in times:
            # print("idx,t", idx, t)
            curr_time_loc = np.where(one_sim_data_u == t)[0]
            if curr_time_loc.size != 0:
                sampled_data[t,0:4] = one_sim_data_u[curr_time_loc[0],:]
            else:
                sampled_data[t,0:4] = sampled_data[t-1,0:4]
                sampled_data[t,0] = t

            # Find mean displacement from prev step
            if t > 0:
                curr_loc = sampled_data[t,2:4]
                prev_loc = sampled_data[t-1,2:4]
                diff = curr_loc - prev_loc
                summed = np.sqrt(diff[0]**2 + diff[1]**2)
                sampled_data[t,4] = summed

        one_users[idx] = sampled_data

    mean_spd_list = []
    for key in one_users:
        usr = one_users[key]
        mean_spd_list += [np.mean(usr[:,4])]
        
    return one_users, mean_spd_list

In [87]:
one,two = get_one_sim_usr()

In [88]:
two

[0.14881413685577516,
 0.08532395948456838,
 0.15300223815828196,
 0.15317484550540952,
 0.17636756185469188,
 0.14158636095892244,
 0.11175716617696856,
 0.10449022166340441,
 0.11312125586159909,
 0.08641175811772173,
 0.16713443201356196,
 0.14847497294125622,
 0.10407920168305124,
 0.12292468195283332,
 0.13214026809478224,
 0.11959063351607815,
 0.11493551244568932,
 0.13203180976292103,
 0.11680206754383517,
 0.11013565717679115,
 0.13802392467440727,
 0.08655419617155516,
 0.1092081556712145,
 0.1359008384820938,
 0.12723734815388538,
 0.1245349542075858,
 0.12105502445823133,
 0.09602427537985254,
 0.135925887001187,
 0.11743289208193047,
 0.10007824916506629,
 0.11961077939481844,
 0.14264552798878127,
 0.11481861276488098,
 0.10399604540125065,
 0.12087420891538826,
 0.12151380993376144,
 0.11252207450586887,
 0.11947539279628157,
 0.08231717961711649,
 0.12432591186189677,
 0.1231351341092658,
 0.10721796063625079,
 0.12307039432774505,
 0.10060973325558617,
 0.1042483175945